##Call the API with my variables for my research

In [146]:
import requests
import json
import numpy as np

# download the Demographics_Native_Country_Of_Mother
key='&key='+'804d0a1a18d1de70764950c78e2a3a42d3d45e48'
data = 'https://api.census.gov/data/2020/cps/basic/jan?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PECYC&row+PEEDUCA&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:31' + key
try:
  data = requests.get(data)
  print(data)
except requests.exceptions.RequestException as e:
    # Handle connection errors or HTTP errors
    print("Tools for Data Science - Semester Project: An error occurred:", e)

An error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


##Display a sample (a couple of lines in the top and some at the end)

In [32]:
import pandas as pd
import regex as re

##Display the first 100 and last 100 characters
data = data.text
print (data[0:100:],'\n')
print (data[-100:])

[["tabulate","state", "PEMNTVTY", "PEMARITL", "PECYC", "PEEDUCA", "PESEX", "PRCITSHP"],
[0,"31","163 

33","5","2","41","1","1"],
[0,"31","168","5","2","41","1","1"],
[0,"31","423","5","2","41","1","1"]]


For some reason the data will return tabulate data with 0, witch means no possible match in regards to all the variables.

In [119]:
##Clean the data
pattern_cleaning = r'\[0,.*?\],?\n?' #remove the data with no tabulate information (tabulate=0)
data_clean = re.sub(pattern_cleaning, '', data)
pattern_cleaning = r'\n' #remove retunr line
data_clean = re.sub(pattern_cleaning, '', data_clean)
data_clean

'[["tabulate","state", "PEMNTVTY", "PEMARITL", "PECYC", "PEEDUCA", "PESEX", "PRCITSHP"],[1414,"31","119","1","-1","44","2","3"],[2381,"31","057","-1","-1","-1","2","3"],[1245,"31","057","1","-1","43","2","3"],[1149,"31","233","1","4","40","1","3"],[1397,"31","303","1","-1","33","2","4"],[1811,"31","313","5","-1","33","2","4"],[1351,"31","207","1","-1","44","2","4"],[1961,"31","313","1","-1","39","2","4"],[1143,"31","301","1","-1","39","2","4"],[3985,"31","303","1","-1","39","2","4"],[1651,"31","303","6","-1","39","2","4"],[1034,"31","151","1","3","42","2","4"],[1731,"31","313","5","-1","31","2","4"],[1309,"31","303","1","1","40","2","4"],[1197,"31","233","1","3","40","2","4"],[1136,"31","416","1","3","40","2","4"],[1375,"31","313","6","3","40","2","4"],[2955,"31","313","-1","-1","-1","2","4"],[1423,"31","313","3","-1","32","2","4"],[1452,"31","207","1","-1","43","2","4"],[1253,"31","210","1","-1","43","2","4"],[1694,"31","303","1","-1","43","2","4"],[1496,"31","220","1","-1","43","2","

In [120]:
##Separe cols from data (numbers)
pattern = r'.*?(\["tabulate".*?"PRCITSHP"\]),([\s\S]*)'
matches = re.search(pattern, data_clean)
data_cols = matches.group(1)
data_vals = matches.group(2)[:-1] #removing the last char "]" at the end.
#print (data_cols)
data_vals

'[1414,"31","119","1","-1","44","2","3"],[2381,"31","057","-1","-1","-1","2","3"],[1245,"31","057","1","-1","43","2","3"],[1149,"31","233","1","4","40","1","3"],[1397,"31","303","1","-1","33","2","4"],[1811,"31","313","5","-1","33","2","4"],[1351,"31","207","1","-1","44","2","4"],[1961,"31","313","1","-1","39","2","4"],[1143,"31","301","1","-1","39","2","4"],[3985,"31","303","1","-1","39","2","4"],[1651,"31","303","6","-1","39","2","4"],[1034,"31","151","1","3","42","2","4"],[1731,"31","313","5","-1","31","2","4"],[1309,"31","303","1","1","40","2","4"],[1197,"31","233","1","3","40","2","4"],[1136,"31","416","1","3","40","2","4"],[1375,"31","313","6","3","40","2","4"],[2955,"31","313","-1","-1","-1","2","4"],[1423,"31","313","3","-1","32","2","4"],[1452,"31","207","1","-1","43","2","4"],[1253,"31","210","1","-1","43","2","4"],[1694,"31","303","1","-1","43","2","4"],[1496,"31","220","1","-1","43","2","4"],[1413,"31","448","1","-1","36","2","4"],[1357,"31","247","1","-1","36","2","4"],[16

In [143]:
data_colsList = re.findall(r'"(.*?)"', data_cols) #Transform string_column in List_column

#Transform string_row in List of Lists then Matrix
rows = re.findall(r'\[(.*?)\]', data_vals)
dataM=[]
for row in rows:
  row_list = re.findall(r'-?\d+(?:\.\d+)?', row)
  dataM.append([int(elt) for elt in row_list])

#print (rows[1]) #TEST
#print (dataM[1]) #TEST

In [144]:
##Convert the result to a Df
df = pd.DataFrame(dataM, columns=data_colsList)
df.head()

,tabulate,state,PEMNTVTY,PEMARITL,PECYC,PEEDUCA,PESEX,PRCITSHP
0,1414,31,119,1,-1,44,2,3
1,2381,31,57,-1,-1,-1,2,3
2,1245,31,57,1,-1,43,2,3
3,1149,31,233,1,4,40,1,3
4,1397,31,303,1,-1,33,2,4


In [11]:
npdata = np.array(data.text)
#npdata.shape
npdata

In [12]:
1+1

2